In [ ]:
#@title 1 - Connect to GPU
#@markdown Run this to connect to a Colab Instance, and see what GPU Google gave you.

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print(gpu[1])
print("The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.")
print("Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.")

Tesla P100-PCIE-16GB
The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.
Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.


In [ ]:
#@title 2 - Mount Google Drive
#@markdown 

import os
%cd /

#Check if Google Drive is mounted, and mount if its not.
googpath = os.path.abspath(os.path.join("gdrive", "My Drive"))
if not os.path.isdir(googpath):
  from google.colab import drive
  drive.mount('/gdrive', force_remount=True)

/
Mounted at /gdrive


In [ ]:
#@title 3 - Import Drive to content
#@markdown 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 4 - Install dependency for VapourSynth and other packages
#@markdown 
!sudo apt-get install python3-pip
!apt update
!apt upgrade

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pkg-resources python3-secretstorage python3-setuptools python3-six
  python3-wheel python3-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python3-cryptography-vectors
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0
  python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pip python3-pkg-resources python3-secretstorage python3-setup

In [ ]:
#@title 4 - Setup {display-mode: "form"}
#@markdown Run this to install VapourSynth, VapourSynth plugins and scripts, as well as some example upscaling models.
#NOTE: running this more than once may or may not work. 
#The buggy console output is due to the threaded installing
#Currently TPU support is broken and incomplete, but it isn't particularly useful since it doesn't support opencl anyway 

#Init
import os, sys, shutil, tempfile
import collections
from datetime import datetime, timedelta
import requests
import threading
import ipywidgets as widgets
from IPython import display
import PIL
from google.colab import files
import time
%cd /

!python3.6 -m pip install -U pip
!python3.6 -m pip install -U setuptools
!python3.6 -m pip install torch torchvision torchaudio cupy-cuda100

#Function defs
#---------------------------------------------------------

#Like shutil.copytree(), but doesn't complain about existing directories
#Note this is fixed in newer version of Python 3
def copytree(src, dst, symlinks=False, ignore=None):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      shutil.copytree(s, d, symlinks, ignore)
    else:
      shutil.copy2(s, d)

#Download and extract the .py scripts from the VapourSynth fatpack
def download_fatpack_scripts():
  %cd /
  print("Downloading VS FatPack Scripts...")
  dlurl = r"https://ipfs.io/ipfs/QmTncsaHZi8eoh4CxLcX6z8c2wwhub5tLDi8pMiM6s6L78/VapourSynth64Portable_2019_11_02.7z"
  with tempfile.TemporaryDirectory() as t:
    dpath = os.path.join(t, "VapourSynth64Portable_2019_11_02.7z")
    os.chdir(t)
    !wget {dlurl}
    %cd /
    !7z x -o{t} {dpath}
    scriptsd = os.path.abspath(os.path.join(t, "VapourSynth64Portable", "Scripts"))
    s = os.path.normpath("VapourSynthImports")
    os.makedirs(s, exist_ok = True)
    copytree(scriptsd, s)
    sys.path.append(s)
  
  #Get some additional scripts.
  !wget -O /VapourSynthImports/muvsfunc_numpy.py https://raw.githubusercontent.com/WolframRhodium/muvsfunc/master/Collections/muvsfunc_numpy.py
  !wget -O /VapourSynthImports/edi_rpow2.py https://gist.githubusercontent.com/YamashitaRen/020c497524e794779d9c/raw/2a20385e50804f8b24f2a2479e2c0f3c335d4853/edi_rpow2.py
  !wget -O /VapourSynthImports/BMToolkit.py https://raw.githubusercontent.com/IFeelBloated/BlockMatchingToolkit/master/BMToolkit.py
  if accelerator == "CUDA":
    !wget -O /VapourSynthImports/Alpha_CuPy.py https://raw.githubusercontent.com/AlphaAtlas/VapourSynth-Super-Resolution-Helper/master/Scripts/Alpha_CuPy.py
    !wget -O /VapourSynthImports/dpid.cu https://raw.githubusercontent.com/WolframRhodium/muvsfunc/master/Collections/examples/Dpid_cupy/dpid.cu
    !wget -O /VapourSynthImports/bilateral.cu https://raw.githubusercontent.com/WolframRhodium/muvsfunc/master/Collections/examples/BilateralGPU_cupy/bilateral.cu

def getdep1():
  %cd /
  #Install apt-fast, for faster installing
  !/bin/bash -c "$(curl -sL https://git.io/vokNn)"
  #Get some basic dependancies
  !apt-fast install -y -q -q subversion davfs2 p7zip-full p7zip-rar ninja-build 

#Get VapourSynth and ImageMagick built just for a colab environment
def getvs():
  %cd /
  #%cd var/cache/apt/archives
  #Artifacts hosted on bintray. If they fail to install, they can be built from source. 
  !curl -L "https://ipfs.io/ipfs/QmTnp2HUsMtV6WKnqCvyCG4nZepQR7MGsPuFFVCyBmptVi/imagemagick_7.0.9-8-1_amd64.deb" -o /var/cache/apt/archives/imagemagick.deb
  !dpkg -i /var/cache/apt/archives/imagemagick.deb
  !ldconfig /usr/local/lib
  !curl -L "https://ipfs.io/ipfs/QmRc92YUXJia8We9PJKs35WZAGhvzoHjbBSFQY7v87pXW7/vapoursynth_48-1_amd64.deb" -o /var/cache/apt/archives/vapoursynth.deb
  !dpkg -i /var/cache/apt/archives/vapoursynth.deb
  !ldconfig /usr/local/lib
  #%cd /

def getvsplugins():
  %cd /
  #Allow unauthenticated sources
  if not os.path.isfile("/etc/apt/apt.conf.d/99myown"):
    with open("/etc/apt/apt.conf.d/99myown", "w+") as f:
      f.write(r'APT::Get::AllowUnauthenticated "true";')
  sources = "/etc/apt/sources.list"
  #Backup original apt sources file, just in case
  with tempfile.TemporaryDirectory() as t:
    tsources = os.path.join(t, os.path.basename(sources))
    shutil.copy(sources, tsources)
    #Add deb-multimedia repo
    #Because building dozens of VS plugins is not fun, and takes forever
    with open(sources, "a+") as f:
      deb = "deb https://www.deb-multimedia.org sid main non-free\n"
      if not "deb-multimedia" in f.read():
        f.write(deb)

    with open(sources, "a+") as f:
      #Temporarily use Debian unstable for some required dependencies 
      if not "ftp.us.debian.org" in f.read():
        f.write("deb http://ftp.us.debian.org/debian/ sid main\n")
    !add-apt-repository -y ppa:deadsnakes/ppa
    !apt-fast update -oAcquire::AllowInsecureRepositories=true
    !apt-fast install -y --allow-unauthenticated deb-multimedia-keyring
    !apt-fast update  

    #Parse plugins to install
    out = !apt-cache search vapoursynth
    vspackages = ""
    #exclude packages with these strings in the name
    exclude = ["waifu", "wobbly", "editor", "dctfilter", "vapoursynth-dev", "vapoursynth-doc"]
    for line in out:
      p =  line.split(" - ")[0].strip()
      if not any(x in p for x in exclude) and "vapoursynth" in p and p != "vapoursynth":
        vspackages = vspackages + p + " "
    print(vspackages)
    #Install VS plugins and a newer ffmpeg build
    !apt-fast install -y --allow-unauthenticated --no-install-recommends ffmpeg youtube-dl libzimg-dev {vspackages} libfftw3-3 libfftw3-double3 libfftw3-dev libfftw3-bin libfftw3-double3 libfftw3-single3 checkinstall
    #Get a tiny example video
    !youtube-dl -o /content/enhance.webm -f 278 https://www.youtube.com/watch?v=I_8ZH1Ggjk0
    #Restore original sources
    os.remove(sources)
    shutil.copy(tsources, sources)
  #Congrats! Apt may or may not be borked.
  copytree("/usr/lib/x86_64-linux-gnu/vapoursynth", "/usr/local/lib/vapoursynth")
  !ldconfig /usr/local/lib/vapoursynth

#Install vapoursynth python modules
def getpythonstuff():
  %cd /
  !/usr/bin/python3.6 -m pip install vapoursynth meson opencv-python

def cudastuff():
    %cd /
    out = !nvcc --version
    cudaver = (str(out).split("Cuda compilation tools, release ")[1].split(", ")[0].replace(".", ""))
    #Note this download sometimes times out
    !/usr/bin/python3.6 -m pip install mxnet-cu{cudaver} #cupy-cuda{cudaver}
    !/usr/bin/python3.6 -m pip install git+https://github.com/AlphaAtlas/VSGAN.git

    #Mxnet stuff
    
    modelurl = "https://github.com/WolframRhodium/Super-Resolution-Zoo/trunk"
    if os.path.isdir("/NeuralNetworks"):
      !svn update --set-depth immediates /NeuralNetworks
      !svn update --set-depth infinity /NeuralNetworks/ARAN
    else:
      !svn checkout --depth immediates {modelurl} /NeuralNetworks

def makesrcd(name):
  %cd /
  srpath = os.path.abspath(os.path.join("/src", name))
  os.makedirs(srpath, exist_ok = False)
  %cd {srpath}

def mesongit(giturl):
  p = os.path.basename(giturl)[:-4]
  makesrcd(p)
  !git clone {giturl}
  %cd {p}
  !meson build
  !ninja -C build
  !ninja -C build install

#Taken from https://stackoverflow.com/a/31614591
#Allows exceptions to be caught from threads
from threading import Thread

class PropagatingThread(Thread):
    def run(self):
        self.exc = None
        try:
            if hasattr(self, '_Thread__target'):
                # Thread uses name mangling prior to Python 3.
                self.ret = self._Thread__target(*self._Thread__args, **self._Thread__kwargs)
            else:
                self.ret = self._target(*self._args, **self._kwargs)
        except BaseException as e:
            self.exc = e

    def join(self):
        super(PropagatingThread, self).join()
        if self.exc:
            raise self.exc
        return self.ret


#Interpolation experiment
#%cd /
#os.makedirs("/videotools")
#%cd /videotools
#!git clone https://github.com/sniklaus/pytorch-sepconv.git
#%cd /

#Function for testing vapoursynth scripts
#Takes the path of the script, and a boolean for generating a test frame.

#-----------------------------------------------------------

#Init functions are threaded for speed
#"PropagatingThread" class is used to return exceptions from threads, otherwise they fail silently

t1 = PropagatingThread(target = getdep1)
t1.start()
print("apt init thread started")

t2 = PropagatingThread(target = download_fatpack_scripts)
t2.start()
print("VS script downloader thread started.")

#Get rid of memory usage log spam from MXnet
os.environ["TCMALLOC_LARGE_ALLOC_REPORT_THRESHOLD"] = "107374182400"

#Check for an accelerator
accelerator = None
gpu = None
if 'COLAB_TPU_ADDR' in os.environ:
  #WIP
  raise Exception("TPUs are (currently) not supported! Please use a GPU or CPU instance.")
else:
  #Check for Nvidia GPU, and identify it 
  out = !command -v nvidia-smi
  if out != []:
    out = !nvidia-smi
    for l in out:
      if "Driver Version" in l:
        accelerator = "CUDA"
        print("Nvidia GPU detected:")
        gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
        gpu = gpu[1]
        #print("Tesla K80 < Tesla T4 < Tesla P100")
        break
if accelerator == None:
  print("Warning: No Accelerator Detected!")

t1.join()
print("Apt init thread done.")

t1 = PropagatingThread(target = getvs)
t1.start()
print("Vapoursynth/Imagemagick downloader thread started.")
t1.join()
print("Vapoursynth/Imagemagick installed")

t3 = PropagatingThread(target = getpythonstuff)
t3.start()
print("Pip thread started")

t1 = PropagatingThread(target = getvsplugins)
t1.start()
print("VS plugin downloader thread started.")

t3.join()
print("pip thread done")

if accelerator == "TPU":
  #WIP!
  pass

elif accelerator == "CUDA":
  t3 = PropagatingThread(target = cudastuff)
  t3.start()
  print("CUDA pip thread started.")
else:
  pass

t2.join()
print("VS script downloader thread done.")

t3.join()
print("CUDA pip thread done.")

t1.join()
print("VS plugin thread done.")



#Build some more plugins(s)
googpath = None
%cd /

Clear_Console_Output_When_Done = True #@param {type:"boolean"}
if Clear_Console_Output_When_Done:
  display.clear_output()
#if gpu is not None:
#  print(gpu[1])
#  print("A Tesla T4 or P100 is significantly faster than a K80")
#  print("And the K80 doesn't support hardware encoding.")


/
    100% |████████████████████████████████| 1.6MB 952kB/s 
  Found existing installation: pip 9.0.1
    Not uninstalling pip at /usr/lib/python3/dist-packages, outside environment /usr
     |████████████████████████████████| 819 kB 7.4 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.0.1
    Uninstalling setuptools-39.0.1:
      Successfully uninstalled setuptools-39.0.1
     |████████████████████████████████| 831.4 MB 10 kB/s 
     |████████████████████████████████| 22.1 MB 90.9 MB/s 
     |████████████████████████████████| 1.9 MB 39.8 MB/s 
     |████████████████████████████████| 59.0 MB 47 kB/s 
     |████████████████████████████████| 14.8 MB 271 kB/s 
     |████████████████████████████████| 3.0 MB 52.9 MB/s 
/
apt init thread started
/
VS script downloader thread started.
Nvidia GPU detected:
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ub

FileNotFoundError: ignored

# Processing

In [ ]:
%%writefile /content/autogenerated.vpy

#This is the Vapoursynth Script!
#Running this cell will write the code in this cell to disk, for VSPipe to read.
#Later cells will check to see if it executes.
#Edit it just like a regular python VS script.
#Search for functions and function reference in http://vsdb.top/, or browse the "VapourSynthImports" folder. 

#Import functions
import sys, os, cv2
sys.path.append('/VapourSynthImports')
import vapoursynth as vs
import vsgan as VSGAN
import mvsfunc as mvf
#import muvsfunc as muf
#import fvsfunc as fvf
import havsfunc as haf
import Alpha_CuPy as ape
import muvsfunc_numpy as mufnp
#import BMToolkit as bm
import G41Fun as G41
#import vsutil as util
#import edi_rpow2 as edi
#import kagefunc as kage
#import lostfunc as lost
#import vsTAAmbk as taa
#import xvs as xvs
from vapoursynth import core

#Set RAM cache size, in MB
core.max_cache_size = 10500

#Get Video(s) or Image(s). ffms2 (ffmpeg) or imwri (imagemagick) will read just about anything.
#Lsmash sometimes works if ffms2 failes, d2v reads mpeg2 files
clip = core.ffms2.Source(r"/content/drive/My Drive/videofile")
#clip = core.ffms2.Source(r"/content/drive/My Drive/upscale training/episodios/video pokemon/064.mkv")
#clip = core.ffms2.Source(r"/content/drive/My Drive/upscale training/episodios/video/[Beatrice-Raws] One Piece 036 [DVDRip 768x576 x264 AC3].mkv")
#clip = core.lsmas.LWLibavSource("/tmp/%d.png")
#clip = core.imwri.Read("testimage.tiff")

#Store source for previewing
src = clip

#Convert to 16 bit YUV for preprocessing
#clip = core.resize.Spline36(clip, format = vs.YUV444P16)

#Deinterlace
#clip = G41.QTGMC(clip, Preset='Medium')

#Mild deblocking
#clip = fvf.AutoDeblock(clip)

#Convert to floating point RGB
clip = mvf.ToRGB(clip, depth = 32)

#Spatio-temportal GPU denoiser. https://github.com/Khanattila/KNLMeansCL/wiki/Filter-description
#clip = core.knlm.KNLMeansCL(clip, a = 8, d = 4, h = 1.4)


preupscale = clip
#Run ESRGAN model. See https://upscale.wiki/wiki/Model_Database
vsgan_device = VSGAN.VSGAN()
vsgan_device.load_model(model=r"/content/drive/My Drive/models/modelname.pht", scale=4)
clip = vsgan_device.run(clip=clip, chunk = False,  pad = 16)

#clip = core.knlm.KNLMeansCL(clip, a = 7, d = 3, h = 1.4)



#Run MXNet model. See the "MXNet" cell.
#Tensorflow models are also supported!
#sr_args = dict(model_filename=r'/NeuralNetworks/ARAN/aran_c0_s1_x4', up_scale=4, device_id=0, block_w=256, block_h=128, is_rgb_model=True, pad=None, crop=None, pre_upscale=False)
#clip = mufnp.super_resolution(clip, **sr_args)

#HQ downscale on the GPU with dpid
#clip = ape.GPU_Downscale(clip, width = 3840, height = 2160)

#Convert back to YUV 444 format/Rec 709 colorspace
#clip = core.resize.Spline36(clip, format = vs.YUV444P16, matrix_s = "709")

#Strong temporal denoiser and stabilizer with the LR as a motion reference clip, for stabilizing.
#prefilter = core.resize.Spline36(preupscale, format = clip.format, width = clip.width, height = clip.height, matrix_s = "709")
#clip = G41.SMDegrain(clip, tr=3, RefineMotion=True, pel = 1, prefilter = prefilter)

#Another CPU denoiser/stabilizer. "very high" is very slow.
#clip = haf.MCTemporalDenoise(clip, settings = "very high", useTTmpSm = True, maxr=4, stabilize = True)

#Stabilized Anti Aliasing, with some GPU acceleration
#clip = taa.TAAmbk(clip, opencl=True, stabilize = 3)

#Example sharpeners that work well on high-res images
#Masks or mvf.limitfilter are good ways to keep artifacts in check
#clip = core.warp.AWarpSharp2(clip)
#clip = G41.NonlinUSM(clip, z=3, sstr=0.28, rad=9, power=1)

#High quality, strong debanding
#clip = fvf.GradFun3(clip, smode = 2)

#Convert back to 8 bit YUV420 for output. 
clip = core.resize.Spline36(clip, format = vs.YUV420P10, matrix_s = "709", dither_type = "error_diffusion")

#Interpolate to double the source framerate
#super = core.mv.Super(inter)
#backward_vectors = core.mv.Analyse(super, isb = True,  overlap=4, search = 3)
#forward_vectors = core.mv.Analyse(super, isb = False, overlap=4, search = 3)
#inter = core.mv.FlowFPS(inter, super, backward_vectors, forward_vectors, num=0, den=0)

#Stack the source on top of the processed clip for comparison
#src = core.resize.Point(src, width = clip.width, height = clip.height, format = clip.format)
#clip = core.std.StackVertical([clip, src])
#Alternatively, interleave the source and slow down the framerate for easy comparison.
#clip = core.std.Interleave([clip, src])
#clip = core.std.AssumeFPS(clip, fpsnum = 24000/1001)

#clip = core.std.SelectEvery(clip=clip, cycle=48, offsets=[0,1])

clip.set_output()

Writing /content/autogenerated.vpy


# 6 - Upscale and Encode


---



In [ ]:
! PYTHONPATH=/usr/local/lib/python3.6/site-packages vspipe -y "/content/autogenerated.vpy" - | ffmpeg -i pipe: -r 25 -vcodec libx265 -preset slow -rc vbr_hq -b:v 105M -maxrate:v 120M -c:a copy -b:a 240k -sn "/content/drive/My Drive/out/filename.mkv"

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --disable-decoder=amrnb --disable-decoder=libopenjpeg --disable-gnutls --disable-liblensfun --disable-libopencv --disable-podpages --disable-sndio --disable-stripping --enable-avfilter --enable-avresample --enable-gcrypt --enable-gpl --enable-ladspa --enable-libaom --enable-libaribb24 --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libdavs2 --enable-libdc1394 --enable-libfdk-aac --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libkvazaar --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librs